In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Modeling imports 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

DATA_DIR = r"C:\Users\ASUS\Downloads\crop_dataset"   

def find_csv_files(data_dir):
    files = [f for f in os.listdir(data_dir) if f.lower().endswith('.csv')]
    files.sort()
    return files

def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except Exception:
        try:
            return pd.read_csv(path, encoding='latin1')
        except Exception as e:
            print(f"Failed to read {path}: {e}")
            return None

def summarize_dfs(dfs):
    summary = []
    for name, df in dfs.items():
        print("\n" + "="*60)
        print(f"File: {name}")
        print("- shape:", df.shape)
        print("\n-- head()")
        print(df.head().to_string(index=False))
        print("\n-- dtypes:")
        print(df.dtypes)
        print("\n-- describe():")
        print(df.describe().T)
        miss = df.isnull().sum()
        miss_pct = miss / len(df) * 100
        miss_df = pd.concat([miss, miss_pct], axis=1)
        miss_df.columns = ['missing_count', 'missing_pct']
        print("\n-- top missing columns:")
        print(miss_df.sort_values('missing_count', ascending=False).head(20))
        summary.append({
            'file': name,
            'shape': df.shape,
            'num_columns': df.shape[1],
            'num_missing_columns': int((miss > 0).sum())
        })
    return pd.DataFrame(summary)

def find_common_columns(dfs):
    all_cols = []
    for df in dfs.values():
        all_cols.extend(list(df.columns))
    common_counts = Counter(all_cols)
    return [col for col, cnt in common_counts.items() if cnt > 1]

def attempt_merge(dfs):
    
    common_cols = find_common_columns(dfs)
    candidate_keys = [c for c in common_cols if c.lower() in ('date','state','district','location','crop','year','month')]
    print("Common columns across files:", common_cols)
    print("Candidate merge keys:", candidate_keys)

    if candidate_keys:
        key = candidate_keys[0]
        print("Merging on key:", key)

        dfs_with_key = {n: df for n, df in dfs.items() if key in df.columns}
        merged = None
        for name, df in dfs_with_key.items():
            if merged is None:
                merged = df.copy()
            else:
                try:
                    merged = pd.merge(merged, df, on=key, how='outer', suffixes=('', f'_{name.split('.')[0]}'))
                except Exception:
                   
                    merged[key] = merged[key].astype(str)
                    df[key] = df[key].astype(str)
                    merged = pd.merge(merged, df, on=key, how='outer', suffixes=('', f'_{name.split('.')[0]}'))
       
        remaining = [n for n in dfs.keys() if n not in dfs_with_key]
        for name in remaining:
            df = dfs[name]
            if merged is not None and len(df) == len(merged):
                merged = pd.concat([merged.reset_index(drop=True), df.reset_index(drop=True)], axis=1)
            else:
                merged = pd.concat([merged, df], ignore_index=True, sort=False)
    else:
        
        groups = {}
        for name, df in dfs.items():
            col_key = tuple(df.columns)
            groups.setdefault(col_key, []).append(df)
        if any(len(v) > 1 for v in groups.values()):
            merged_parts = []
            for k, v in groups.items():
                if len(v) == 1:
                    merged_parts.append(v[0])
                else:
                    merged_parts.append(pd.concat(v, ignore_index=True, sort=False))
            merged = pd.concat(merged_parts, ignore_index=True, sort=False)
        else:
            merged = pd.concat(dfs.values(), ignore_index=True, sort=False)
    return merged

def save_df(df, path):
    df.to_csv(path, index=False)
    print(f"Saved: {path}")

def plot_missing_values(df, outpath):
    miss = df.isnull().sum().sort_values(ascending=False).head(20)
    plt.figure(figsize=(10,4))
    plt.barh(miss.index[::-1], miss.values[::-1])
    plt.title("Top 20 columns by missing values")
    plt.xlabel("Missing values")
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()
    print("Saved:", outpath)

def plot_correlation_heatmap(df, outpath):
    numeric = df.select_dtypes(include=[np.number])
    if numeric.shape[1] < 2:
        print("Not enough numeric columns for correlation heatmap.")
        return
    corr = numeric.corr()
    top_cols = corr.abs().sum().sort_values(ascending=False).head(12).index
    M = numeric[top_cols].corr()
    plt.figure(figsize=(8,6))
    im = plt.imshow(M, interpolation='none', cmap='viridis')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(range(len(top_cols)), top_cols, rotation=45, ha='right')
    plt.yticks(range(len(top_cols)), top_cols)
    plt.title("Correlation heatmap (top numeric columns)")
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()
    print("Saved:", outpath)

def attempt_modeling(merged_df, data_dir):
    yield_cols = [c for c in merged_df.columns if 'yield' in c.lower()]
    if not yield_cols:
        print("No yield-like column found, skipping modeling.")
        return
    target_col = yield_cols[0]
    print("Using target:", target_col)
    df = merged_df.copy()
    y = df[target_col]
    X = df.drop(columns=[target_col])
    X_num = X.select_dtypes(include=[np.number]).copy()

    if X_num.shape[1] == 0:
        print("No numeric features available for modeling.")
        return
    
    thresh_missing = 0.6 * len(X_num)
    to_keep = [c for c in X_num.columns if X_num[c].isnull().sum() <= thresh_missing]
    X_num = X_num[to_keep]
    
    mask = y.notnull()
    X_num = X_num.loc[mask]
    y_model = y.loc[mask]
    if X_num.shape[0] < 30:
        print("Not enough rows with non-null target for reliable modeling. Rows:", X_num.shape[0])
        return
    # Simple pipeline
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
    ])
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X_num, y_model, cv=cv, scoring='r2', n_jobs=1)
    print("Cross-validated R2 scores:", scores)
    print("Mean R2:", scores.mean())
    # Fit and get feature importances
    pipeline.fit(X_num, y_model)
    model = pipeline.named_steps['model']
    try:
        importances = pd.Series(model.feature_importances_, index=X_num.columns).sort_values(ascending=False)
        fi_path = os.path.join(data_dir, 'feature_importances.csv')
        importances.to_csv(fi_path, header=['importance'])
        print("Saved feature importances to:", fi_path)
        print(importances.head(10).to_string())
    except Exception as e:
        print("Model does not support feature_importances_ or failed to extract:", e)

def main():
    csv_files = find_csv_files(DATA_DIR)
    if not csv_files:
        print("No CSV files found in", DATA_DIR)
        return
    print("Found CSV files:", csv_files)
    dfs = {}
    for f in csv_files:
        path = os.path.join(DATA_DIR, f)
        df = safe_read_csv(path)
        if df is not None:
            dfs[f] = df
        else:
            print("Skipping file (could not read):", f)

    # Summarize
    summary_df = summarize_dfs(dfs)
    print("\nSummary table:")
    print(summary_df.to_string(index=False))

    # Merge
    merged = attempt_merge(dfs)
    if merged is None:
        print("Merge failed or produced no data.")
        return
    print("\nMerged shape:", merged.shape)
    print(merged.head().to_string(index=False))

    # Save merged
    merged_path = os.path.join(DATA_DIR, 'merged_data.csv')
    save_df(merged, merged_path)

    # EDA plots
    plot_missing_values(merged, os.path.join(DATA_DIR, 'missing_values_top20.png'))
    plot_correlation_heatmap(merged, os.path.join(DATA_DIR, 'correlation_heatmap.png'))

    # Attempt modeling
    attempt_modeling(merged, DATA_DIR)

    print("\nDone. Check /mnt/data for outputs (merged_data.csv, plots, feature_importances.csv if any).")

if __name__ == "__main__":
    main()


Found CSV files: ['pesticides.csv', 'rainfall.csv', 'temp.csv', 'yield.csv', 'yield_df.csv']

File: pesticides.csv
- shape: (4349, 7)

-- head()
        Domain    Area Element               Item  Year                         Unit  Value
Pesticides Use Albania     Use Pesticides (total)  1990 tonnes of active ingredients  121.0
Pesticides Use Albania     Use Pesticides (total)  1991 tonnes of active ingredients  121.0
Pesticides Use Albania     Use Pesticides (total)  1992 tonnes of active ingredients  121.0
Pesticides Use Albania     Use Pesticides (total)  1993 tonnes of active ingredients  121.0
Pesticides Use Albania     Use Pesticides (total)  1994 tonnes of active ingredients  201.0

-- dtypes:
Domain      object
Area        object
Element     object
Item        object
Year         int64
Unit        object
Value      float64
dtype: object

-- describe():
        count          mean            std     min     25%      50%     75%  \
Year   4349.0   2003.138883       7.728044  1990.

MemoryError: Unable to allocate 7.37 GiB for an array with shape (989712018,) and data type int64